In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def importing_datasets():
    file_path = "/data/work/shared/safe_data/sleep_skew/skew_project_start_oct_2022/"
    phone_sample = pd.read_csv(file_path+"phone_data_s60.csv")
    phone_sample['day_label_dt'] = pd.to_datetime(phone_sample['day_label'],unit='d')

    activity_sample = pd.read_csv(file_path+"steps_data_s60.csv")
    activity_sample['day_label_dt'] = pd.to_datetime(activity_sample['day_label'],unit='d')


    '''Getting the sleeping info of the chosen user from another file'''
    sleep_sample = pd.read_csv(file_path+"sleep_skew_data_w_demographics.csv")
    sleep_sample['sleep_duration_round'] = np.round(sleep_sample['sleep_duration']/3600,2)
    sleep_sample['sleep_start'] = pd.to_datetime(sleep_sample['day_label'],unit='d')
    sleep_sample['sleep_start_shift'] = np.where(sleep_sample['start_time_num_round'] >= 12,
                                                sleep_sample['sleep_start'],
                                                 sleep_sample['sleep_start']-pd.to_timedelta(1,unit='d')
                                                )
    sleep_sample['day_of_week'] = sleep_sample['sleep_start_shift'].dt.day_name()
    sleep_sample['weekend'] = np.where(sleep_sample['day_of_week'].isin(['Friday','Saturday']),1,0)

    sleep_sample['start_time_num_round_25'] = ((sleep_sample['start_time_num_round']%1)*100/60)+sleep_sample['start_time_num_round'].astype(int)
    sleep_sample['end_time_num_round_25'] = ((sleep_sample['end_time_num_round']%1)*100/60)+sleep_sample['end_time_num_round'].astype(int)

    '''EXCEPTION HANDLING FOR 24.00 : SHOULD SHOW AS 00.00'''
    
    sleep_sample['start_time_num_round_25'] = round(sleep_sample['start_time_num_round_25']*4)/4
    sleep_sample['end_time_num_round_25'] = round(sleep_sample['end_time_num_round_25']*4)/4
    
    sleep_sample['start_time_num_round_25'] = np.where(sleep_sample['start_time_num_round_25']==24,0,sleep_sample['start_time_num_round_25'])
    sleep_sample['end_time_num_round_25'] = np.where(sleep_sample['end_time_num_round_25']==24,0,sleep_sample['end_time_num_round_25'])

    ''' EXCEPTION HANDLING FOR SLEEP ONSET BETWEEN 23.45 AND 23.59 : SHOW AS 23.75 '''
    sleep_sample['start_time_num_round_25'] = np.where(((sleep_sample['start_time_num_round'] >= 23.45) &
                                                       (sleep_sample['start_time_num_round'] < 24.0)), 23.75,sleep_sample['start_time_num_round_25'])
    sleep_sample['end_time_num_round_25'] = np.where(((sleep_sample['end_time_num_round'] >= 23.45) &
                                                      (sleep_sample['end_time_num_round'] < 24.0)), 23.75,sleep_sample['end_time_num_round_25']) 

    ''' 
    Defining Naps and shift sleep
    If someone sleeps between 11am and 3pm
        - Less than 3 hours : Nap
        - More than 3 hours : Shift worker sleep
    '''
    sleep_sample['nap'] = np.where(
        ((sleep_sample['sleep_duration_round']<3) & 
         (sleep_sample['start_time_num_round'] < 15) & 
         (sleep_sample['start_time_num_round'] > 11)),1,0)
    sleep_sample['shift_sleep'] = np.where(
        ((sleep_sample['sleep_duration_round']>=3) & 
         (sleep_sample['start_time_num_round'] < 15) & 
         (sleep_sample['start_time_num_round'] > 11)),1,0)
    
#     sleep_sample['shift_sleep'] = np.where(
#         ((sleep_sample['sleep_duration_round']>=2) & 
#          (sleep_sample['start_time_num_round'] < 15) & 
#          (sleep_sample['start_time_num_round'] > 4)),1,0)

    '''Deleting shift sleeps'''
    sleep_sample = sleep_sample[sleep_sample['shift_sleep'] ==0]

    sleep_sample.drop(columns={'gender','age_group','country','weekday','day_label','day_label_shift'},inplace=True)
    sleep_sample.sort_values(by=['user_id','sleep_start_shift','sleep_start','start_time_num_round'],inplace=True)
    print("Number of rows: ",len(sleep_sample))
    print("Unique users: ",len(sleep_sample['user_id'].unique()))
    
    '''Identifying the unique users in phone and activity data and excluding customers from analysis who dont have
    this data'''
    unique_phone_users = phone_sample['user_id'].unique()
    unique_activity_users = activity_sample['user_id'].unique()
    sleep_sample = sleep_sample[(sleep_sample['user_id'].isin(unique_phone_users)) & 
                                (sleep_sample['user_id'].isin(unique_activity_users))]

#     print("Count of users in sleep data: ",len(sleep_sample['user_id'].unique()))

    '''need to make a mapping between alpha numeric user ids and integer user_ids'''
    user_id_mapping = pd.DataFrame(sleep_sample['user_id'].unique()).reset_index()
    user_id_mapping.columns=['new_user_id','user_id']
    user_id_mapping['new_user_id'] = user_id_mapping['new_user_id']+1
#     print(np.min(user_id_mapping['new_user_id']),np.max(user_id_mapping['new_user_id']))

    def append_userid(df):
        df= df.merge(user_id_mapping,how='left',left_on = 'user_id',right_on='user_id')
        df.drop(columns={'user_id'},inplace=True)
        df.rename(columns={'new_user_id':'user_id'},inplace=True)
        return df

    sleep_sample = append_userid(sleep_sample)
    phone_sample = append_userid(phone_sample)
    activity_sample = append_userid(activity_sample)

#     print("sleep data: ",sleep_sample.columns)
#     print("phone usage: ",phone_sample.columns)
#     print("activity: ",activity_sample.columns)
    
    return sleep_sample, activity_sample, phone_sample, user_id_mapping
# print("Unique countries: ",len(sleep_sample['country'].unique()))

In [4]:
def minimumcriteria(_sample,column_name):
    day_rollup = _sample[['user_id','day_label',column_name]].groupby(['user_id','day_label']).sum().reset_index()

    '''will include only those days where phone viewing per day is more than 60 minutes'''
    if column_name == 'duration': #minimum duration cut off for phone usage in a day
        day_rollup = day_rollup[day_rollup[column_name]>=3600]
    else: # minimum cut off for steps in a day
        day_rollup = day_rollup[day_rollup[column_name]>=1000]
    day_rollup.sort_values(by=['user_id','day_label'],inplace=True)
    
    _sample = _sample.merge(day_rollup[['user_id','day_label']],how='inner',
                            left_on = ['user_id','day_label'],
                           right_on = ['user_id','day_label'])
    return _sample
    
    
    

In [5]:
def sleep_streaks(sleep_sample):
    sleep_sample['lag_sleep_start_shift'] = sleep_sample['sleep_start_shift'].shift(1) #changed from -1
    sleep_sample['lag_user_id'] = sleep_sample['user_id'].shift(1) #changed from -1

    conditions = [
        ((sleep_sample['sleep_start_shift']-sleep_sample['lag_sleep_start_shift']).dt.days <= 1) &
        (sleep_sample['lag_user_id'] == sleep_sample['user_id']) & (sleep_sample['nap'] ==0)] # change from ==1,switched around vars in subtraction
    choices = [1]
    sleep_sample['continuous_data_sleep'] = np.select(conditions,choices)

    s = sleep_sample['continuous_data_sleep'].eq(True)
    sleep_sample['count'] = (sleep_sample.groupby(['user_id',s.ne(s.shift()).cumsum()])
                             .cumcount()
                             .add(1)
                            )
    sleep_sample_ContOnly = sleep_sample[sleep_sample['continuous_data_sleep']==1]

    sleep_sample_ContOnly['ContNapCount'] = sleep_sample_ContOnly.groupby(['user_id',s.ne(s.shift()).cumsum()])['nap'].cumsum()
    #keep only first records and records where count > 7
    sleep_sample_ContOnly = sleep_sample_ContOnly[(sleep_sample_ContOnly['count']==1) | (sleep_sample_ContOnly['count']>30)]
    print("len sleep_sample_ContOnly: ",len(sleep_sample_ContOnly))
    
    SleepContInnings = []
    for i in range(2,len(sleep_sample_ContOnly)):
            if sleep_sample_ContOnly['count'].iloc[i] == 1:
                SleepContInnings.append([sleep_sample_ContOnly['user_id'].iloc[i-1],
                                         sleep_sample_ContOnly['sleep_start_shift'].iloc[i-1],
                                         sleep_sample_ContOnly['count'].iloc[i-1],
                                             sleep_sample_ContOnly['ContNapCount'].iloc[i-1]])
    #adding the last streak, however short or long it is
    SleepContInnings.append([sleep_sample_ContOnly['user_id'].iloc[-1],
                                         sleep_sample_ContOnly['sleep_start_shift'].iloc[-1],
                                         sleep_sample_ContOnly['count'].iloc[-1],
                                             sleep_sample_ContOnly['ContNapCount'].iloc[-1]])            
    #     print(len(SleepContInnings))
    SleepContInnings_df = pd.DataFrame(SleepContInnings,columns =['user_id','sleep_start_shift','count','ContNapCount'])
    SleepContInnings_df = SleepContInnings_df[SleepContInnings_df['count']>1]
    SleepContInnings_df['sleep_streak_start_date']=SleepContInnings_df['sleep_start_shift']-pd.to_timedelta(SleepContInnings_df['count']-1,unit='d')

    SleepContInnings_df = SleepContInnings_df.merge(_skew,how='left',left_on='user_id',right_on='user_id')
    return sleep_sample, SleepContInnings_df, sleep_sample_ContOnly
#     SleepContInnings_df.head()

In [6]:
def calculating_streaks(phone_sample, column_name):
    phone_user_day_rollup = phone_sample[['user_id','day_label',column_name]].groupby(['user_id','day_label']).sum().reset_index()

    '''will include only those days where phone viewing per day is more than 60 minutes'''
    if column_name == 'duration': #minimum duration cut off for phone usage in a day
        phone_user_day_rollup = phone_user_day_rollup[phone_user_day_rollup[column_name]>=3600]
    else: # minimum cut off for steps in a day
        phone_user_day_rollup = phone_user_day_rollup[phone_user_day_rollup[column_name]>=1000]
    phone_user_day_rollup.sort_values(by=['user_id','day_label'],inplace=True)
    
    phone_user_day_rollup['lag_day_label'] = phone_user_day_rollup['day_label'].shift(1) #changed from -1
    phone_user_day_rollup['lag_user_id'] = phone_user_day_rollup['user_id'].shift(1) #changed from -1

    conditions = [
        ((phone_user_day_rollup['day_label']-phone_user_day_rollup['lag_day_label']) <= 1) &
        (phone_user_day_rollup['lag_user_id'] == phone_user_day_rollup['user_id'])] # change from ==1,switched around vars in subtraction
    choices = [1]
    phone_user_day_rollup['continuous_data_phone'] = np.select(conditions,choices)

    s = phone_user_day_rollup['continuous_data_phone'].eq(True)
    phone_user_day_rollup['count'] = (phone_user_day_rollup.groupby(['user_id',s.ne(s.shift()).cumsum()])
                             .cumcount()
                             .add(1)
                            )
    phone_sample_ContOnly = phone_user_day_rollup[phone_user_day_rollup['continuous_data_phone']==1]
    
    PhoneContInnings = []
    for i in range(2,len(phone_sample_ContOnly)):
        if phone_sample_ContOnly['count'].iloc[i] == 1:
            PhoneContInnings.append([phone_sample_ContOnly['user_id'].iloc[i-1],
                                     phone_sample_ContOnly['day_label'].iloc[i-1],
                                     phone_sample_ContOnly['count'].iloc[i-1]])

    PhoneContInnings.append([phone_sample_ContOnly['user_id'].iloc[-1],
                             phone_sample_ContOnly['day_label'].iloc[-1],
                             phone_sample_ContOnly['count'].iloc[-1]])
                            
    #     print(len(PhoneContInnings))
    PhoneContInnings_df = pd.DataFrame(PhoneContInnings,columns =['user_id','day_label','count'])
    PhoneContInnings_df = PhoneContInnings_df[PhoneContInnings_df['count']>1]
    PhoneContInnings_df['day_label_dt']= pd.to_datetime(PhoneContInnings_df['day_label'],unit='d')
    PhoneContInnings_df['phone_streak_start_date']=PhoneContInnings_df['day_label_dt']-pd.to_timedelta(PhoneContInnings_df['count']-1,unit='d')
    PhoneContInnings_df.rename(columns={'day_label_dt':'phone_streak_end_date'},inplace=True)
    return PhoneContInnings_df


In [7]:
def CombinePhoneActivityForSleepStreaks(chosen_user, date_sleep_combo, date_phone_combo, date_activity_combo, 
                                        streak_start, streak_end):

    date_phone_activity_combo = date_sleep_combo.merge(date_phone_combo[['sleep_start','start_time_num_round','duration']],
                                                       how='left',
                                                       left_on = ['sleep_start','start_time_num_round'],
                                                      right_on = ['sleep_start','start_time_num_round'])
        
    date_phone_activity_combo = date_phone_activity_combo.merge(date_activity_combo[['sleep_start','start_time_num_round','steps']],
                                                                how='left',
                                                                left_on = ['sleep_start','start_time_num_round'],
                                                                right_on = ['sleep_start','start_time_num_round'])
    
    # drop the rows where both phone and activity are NULL
#     print("initial count of rows: ", len(date_phone_activity_combo))
    date_phone_activity_combo = date_phone_activity_combo[~((date_phone_activity_combo['duration'].isnull()) &
                                                            (date_phone_activity_combo['steps'].isnull()))]
#     print("final count of rows: ", len(date_phone_activity_combo))
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    a = np.array(date_phone_activity_combo['duration']).reshape(-1,1)
    a1 = scaler.fit_transform(a)
    a1 = pd.Series(a1.flatten(),name='dur_norm')
    date_phone_activity_combo = pd.concat([date_phone_activity_combo.reset_index(drop=True),a1.reset_index(drop=True)],axis=1)
    date_phone_activity_combo['dur_norm'] = date_phone_activity_combo['dur_norm']*10000
#     print("length check : ", len(date_phone_activity_combo))
    a = np.array(date_phone_activity_combo['steps']).reshape(-1,1)
    a1 = scaler.fit_transform(a)
    a1 = pd.Series(a1.flatten(),name='steps_norm')
    date_phone_activity_combo = pd.concat([date_phone_activity_combo.reset_index(drop=True),a1.reset_index(drop=True)],axis=1)
#     print("length check : ", len(date_phone_activity_combo))
    date_phone_activity_combo['steps_norm'] = date_phone_activity_combo['steps_norm']*10000
    
    date_phone_activity_combo['duration_steps_combo'] = np.max(date_phone_activity_combo[['dur_norm','steps_norm']],axis = 1)
    date_phone_activity_combo['duration_steps_combo'] = np.where(date_phone_activity_combo['asleep_0_awake_300'] == 0,
                                                                 0,
                                                                 date_phone_activity_combo['duration_steps_combo'])
                                                                 
#     print("date ph act combo columns: ",date_phone_activity_combo.columns)
    return date_phone_activity_combo

In [8]:
def sleep_nights(chosen_user, sleep_sample, streak_start, streak_length):
    import math
    chosen_user_streak_end =  chosen_user_streak_start+np.timedelta64(chosen_user_streak_length, 'D')
    cc = sleep_sample[(sleep_sample["user_id"]==chosen_user) &
                      (sleep_sample["sleep_start"]>=  chosen_user_streak_start) &
                      (sleep_sample["sleep_start"] <=  chosen_user_streak_end)]
    cc = cc.reset_index()
    cc.drop(columns={"index"},inplace=True)
    '''rounding the start and end times of sleep to the nearest quarter for ease of calculation. will preserve
    sleep duration calculation'''

#     print("#sleep records before dropping multiple sleeps: ",len(cc))
    cc.sort_values(by=['sleep_start_shift','sleep_start','start_time_num_round'],
                   ascending=[True,True,True],inplace=True)
    cc.drop_duplicates(subset = ['sleep_start_shift','nap'],inplace=True)

    cc= cc[['user_id','sleep_start','sleep_start_shift','day_of_week','weekend',
            'start_time_num_round_25','end_time_num_round_25','sleep_duration','sleep_duration_round','nap',
            'shift_sleep','count','lag_sleep_start_shift','continuous_data_sleep','lag_user_id']]
#     print("#sleep records AFTER dropping multiple sleeps: ",len(cc))
    
    return cc
    
    '''Step II: Creating date sleep combinations'''
    
def drop_dates(cc):
    '''
    IF the first night of sleep has the same day label and shift label i.e. it starts in the same clock hours
    of the day label ( for eg day_label = 17025, shift = 17025, sleep_start_time = 23.5) then that DAY should be EXCLUDED from the analysis as we dont have the whole day's data,
    we cannot assume AWAKENESS at the early hours of that day as technically there wont be a sleep record 
    ( for instance, record of 00.00 to 6.00 of 17025). Consequently, sleep prediction should start from 
    the NEXT day ie for 17026 so that model is analyzing full day behavior of 17025 and predicting DLMO 
    onset for 17026. Predictions are made for shifted labels.

    In any case, prediction of DLMO onset should only begin from the NEXT shifted label)
    '''
    date_to_be_dropped = []
    if cc['sleep_start'].iloc[0] == cc['sleep_start_shift'].iloc[0]: #same clock hour night sleep onset
        date_to_be_dropped.append(cc['sleep_start'].iloc[0])

    if cc['sleep_start'].iloc[0] != cc['sleep_start_shift'].iloc[0]: #different dates so drop the shifted date
        date_to_be_dropped.append(cc['sleep_start_shift'].iloc[0])

    if cc['sleep_start'].iloc[-1]  != cc['sleep_start_shift'].iloc[-1]:
        date_to_be_dropped.append(cc['sleep_start'].iloc[-1])
#     print("Dates to be dropped:",date_to_be_dropped)
    return date_to_be_dropped


def create_combo(dd,date_to_be_dropped,column_name):

    time_vals = np.arange(0.0, 23.75, 0.25)
    time_vals = np.append(time_vals,23.75)
    
    if isinstance(dd,pd.DataFrame):

        unique_dates = np.unique(dd[column_name])
#         print("Initial unique dates length: ",len(unique_dates))
        unique_dates = [x for x in unique_dates if x not in date_to_be_dropped]
#         print("Initial unique dates length: ",len(unique_dates))

        import itertools
        date_combo =pd.DataFrame(list(itertools.product(np.unique(unique_dates),time_vals)))
        date_combo.columns = [column_name, "start_time_num_round_25"]
        date_combo['start_time_num_round'] = date_combo['start_time_num_round_25'].astype(int) + \
        ((date_combo['start_time_num_round_25']%1)*60)/100
        return date_combo, time_vals
    
    '''STEP III: Plotting distribution'''

def plot_dist(sleep_sample, chosen_user):
    
    from scipy.stats import skew
    sleep_durations = sleep_sample[sleep_sample['user_id'] == chosen_user]['sleep_duration_round']
    _skew= np.round(skew(sleep_durations),2)
    _med= np.round(np.median(sleep_durations),2)
    _avg = np.round(np.mean(sleep_durations),2)
    _skew2 = np.round(_med-_avg,2)
    print(_skew, _med)
    plt.figure(figsize=(10,4))
    plt.hist(sleep_durations, histtype='step',color='brown')
    title_text = "User: "+str(chosen_user)+" sleep duration dist.:: Median:"+str(_med)+" Skew: "+str(_skew)+" Skew 2: "+str(_skew2)
    plt.title(title_text)
    plt.xlabel("Sleep duration in Hours")
#     plt.savefig("/file/out/antar/_user_1104_hist.png",dpi=400)
    plt.show()


In [9]:
def event_times(cc):
    from itertools import cycle
# time_vals
    pool = cycle(time_vals)
    l= [] 
    c=0
    for n in pool:
        if c >=48*4 :
            break
        else:
            l.append(n)
            c+=1

    l[90:100]
    
    '''
setting up two examples:

03-19-2016 : 0.00 to 8.25
03-19-2016 : 22.75 to 8.0
'''

    aa = [] # initializing empty list to hold sleep start shift and .25 hour time intervals of sleep
    for i in range(len(cc)): # for each sleep entry
        first_index = l.index(cc.iloc[i]["start_time_num_round_25"])

        if cc.iloc[i]["start_time_num_round_25"] > cc.iloc[i]["end_time_num_round_25"]: #start time is at night and wakes up next morning
            last_index =  l.index(cc.iloc[i]["end_time_num_round_25"])+ (24*4)
            counter = first_index

            while counter < last_index:
                if (l[counter] >= 0) & (l[counter] <  cc.iloc[i]["start_time_num_round_25"]):
                    aa.append([cc.iloc[i]["sleep_start"]+np.timedelta64(1, 'D'),l[counter]])

                else:
                    aa.append([cc.iloc[i]["sleep_start"],l[counter]])

                counter+=1

        else:
            last_index =  l.index(cc.iloc[i]["end_time_num_round_25"])
            counter = first_index

            while counter < last_index:
                aa.append([cc.iloc[i]["sleep_start"], l[counter]])

                counter+=1    
    sleep_times = pd.DataFrame(aa)
    sleep_times.columns = ["sleep_start","start_time_num_round_25"]
    sleep_times["user_id"] = chosen_user
    return sleep_times


In [10]:
def SleepingDates(cc2):


    sleeping = np.array(cc2[(cc2['user_id'] == chosen_user) &(cc2['nap']==0)]['start_time_num_round_25'])
    sleeping_naps = np.array(cc2[(cc2['user_id'] == chosen_user) &(cc2['nap']==1)]['start_time_num_round_25'])
    sleeping_48hour = []
    for sleeping_time in sleeping:
        if sleeping_time < 12.00:
            sleeping_48hour.append(sleeping_time+24.00)
        else:
            sleeping_48hour.append(sleeping_time) 

    sleeping_48hour_naps = []
    for sleeping_time in sleeping_naps:
        if sleeping_time < 12.00:
            sleeping_48hour_naps.append(sleeping_time+24.00)
        else:
            sleeping_48hour_naps.append(sleeping_time) 


    SleepingDates = cc2[(cc2['user_id'] == chosen_user) &(cc2['nap']==0)]['sleep_start_shift']
    SleepingDatesActual = cc2[(cc['user_id'] == chosen_user) &(cc2['nap']==0)]['sleep_start']
    SleepingDates_naps = cc2[(cc['user_id'] == chosen_user) &(cc2['nap']==1)]['sleep_start_shift']
    sleep_duration = cc2[(cc2['user_id'] == chosen_user) &(cc2['nap']==0)]['sleep_duration_round']

    SleepingDates_48hr = pd.DataFrame(zip(SleepingDates,SleepingDatesActual, sleeping_48hour,sleep_duration),
                                      columns=['sleep_start_shift','sleep_start',
                                               'start_time_num_round_25','sleep_duration_round']) #converting to df


    weekend = np.array(cc2[(cc2['user_id'] == chosen_user) &(cc['nap']==0)]['weekend'])
    return SleepingDates_48hr, weekend


In [11]:
# x1_x2_dlmo_df = pd.read_csv("/data/work/shared/s210142/datasets/x1_x2_dlmo.csv")
# x1_x2_dlmo_df['sleeponset'] = np.round(x1_x2_dlmo_df['dlmo']+2.0,2)
# x1_x2_dlmo_df.drop(columns={'Unnamed: 0'},inplace=True)
# x1_x2_dlmo_df['diff'] = abs(x1_x2_dlmo_df['sleeponset']-SleepingDates_48hr.loc[0, 'start_time_num_round_25'])
# x1 = x1_x2_dlmo_df[x1_x2_dlmo_df['diff']==np.min(x1_x2_dlmo_df['diff'])]['x1'].values[0]
# x2 = x1_x2_dlmo_df[x1_x2_dlmo_df['diff']==np.min(x1_x2_dlmo_df['diff'])]['x2'].values[0]

In [12]:
from scipy.signal import find_peaks
def initial_conditions_loop(SleepingDates_48hr, ts: np.ndarray, #Array of time points, also determines step size of RK4 solver
                                light_est: np.ndarray, #Array of light estimates, should be the same length as ts
                                num_loops: int = 30) -> np.ndarray: 
        """ 
            Estimate the starting values by looping the given light_estimate, commonly used for to estimate the initial conditions
            assumes the individual lives the same schedule repeatedly
        """
#         x1_x2_dlmo_df = pd.read_csv("/data/work/shared/s210142/datasets/x1_x2_dlmo_actual.csv")
#         x1_x2_dlmo_df['sleeponset'] = np.round(x1_x2_dlmo_df['dlmo']+2.0,2)
#         x1_x2_dlmo_df.drop(columns={'Unnamed: 0'},inplace=True)
#         x1_x2_dlmo_df['diff'] = abs(x1_x2_dlmo_df['sleeponset']-SleepingDates_48hr.loc[0, 'start_time_num_round_25'])
#         x1 = x1_x2_dlmo_df[x1_x2_dlmo_df['diff']==np.min(x1_x2_dlmo_df['diff'])]['x1'].values[0]
#         x2 = x1_x2_dlmo_df[x1_x2_dlmo_df['diff']==np.min(x1_x2_dlmo_df['diff'])]['x2'].values[0]
        
        y = np.array([-0.3,-1.13,0.0])
#         print("x1: ",x1," x2: ",x2)
#         y = np.array([x1, x2, 0.0])
        #ic = self.default_initial_conditions
        for _ in range(num_loops):
            sol = integrate_model(ts, light_est, y)
            y = sol[..., -1]
        return y,sol

def integrate_model(ts: np.array,  # Array of time points, also determines step size of RK4 solver
                    light_est: np.array,  # Array of light estimates, should be the same length as ts
                    state: np.array,  # Initial state of the model
                    ):
    nloop = len(ts)
#     print("n: ",nloop)
    sol = np.zeros([state.shape[0], nloop])
    sol[..., 0] = state
#     print(sol[...,0])
    for idx in range(1, nloop):
        state = step_rk4(
        state=state,
        light_val=light_est[idx],
        dt=ts[idx]-ts[idx-1])
        sol[..., idx] = state
#     print("sol shape: ",sol.shape)
    return sol

def step_rk4(state: np.ndarray, #dy/dt = f(y, t)
              light_val: float, #light value at time t in lux
              dt=float #step size in hours
            ):
    k1 = derv(state, light=light_val)
    k2 = derv(state + k1 * dt / 2.0, light=light_val)
    k3 = derv(state + k2 * dt / 2.0, light=light_val)
    k4 = derv(state + k3 * dt, light=light_val)
    state = state + (dt / 6.0) * (k1 + 2.0*k2 + 2.0*k3 + k4)
    #   print("state: ",state)
    return state

def derv(y: np.array, #state vector (x, xc, n)
        light: float, #light value in lux
        ):

        taux = 24.2;
        G = 19.875;
        b = 0.013;
        k = .55;
        
        mu = .1300;
        q = 1.0/3;
        
        I0 = 9500;
        p = .6;
        a0 = 0.16;
        
        # Relational parameters to circadian markers
        phi_ref = 0.80; # relative to x min to get CBTmin, units of hours
        dlmo_convert = -7.0 #factor to convert from CBTmin to DLMO
        
        x =y[0]
        xc = y[1]
        n = y[2]
        #print("x: ",x," xc: ",xc, " n: ",n)
        
        alpha = a0*(light/I0)**p;
        Bh = G*alpha*(1-n);
        B = Bh*(1-0.4*x)*(1-0.4*xc);
               
        dydt = np.zeros_like(y)
        dydt[0] = np.pi/12* (xc + mu*(1/3*x+4/3*x**3-256/105*x**7) + B);
        dydt[1] = np.pi/12* (q*B*xc - x*((24/(0.99729*taux))**2 + k*B));
        dydt[2] = 60*(alpha*(1-n) - b*n);
        return dydt
    
def cbt(sol,show_plot):
        phi_ref = 0.8
        cbt_mins_test = find_peaks(-1*sol[0,:])[0]
        cbt_mins = find_peaks(-1*sol[0,:], distance =60 )[0] # min of x is the CBTmin
        # height condition so that peaks in the opposite direction are not considered, distance to ensure peaks are not chosen too close
        cbts = sol[0,:]
#         print("cbts: ",len(cbts))
        if show_plot == 'yes':
            plt.figure(figsize=(16,4))
            plt.xlabel("Day labels")
            plt.ylabel("cbts")
            plt.plot(cbts)
            plt.plot(cbt_mins_test, cbts[cbt_mins_test], "x",color = "red")
            plt.title("CBT signal and identified peaks without distance constraints")
            plt.show()


            plt.figure(figsize=(16,4))
            plt.xlabel(" $t$ (Quarter of an hour)",fontsize=12)
            plt.ylabel("cbts",fontsize=12)
            plt.plot(cbts)
            plt.plot(cbt_mins, cbts[cbt_mins], "x",color = "red")
            plt.title("CBT signal and identified peaks")
            plt.show()
#         print("length cbt mins:",cbt_mins.shape)
#         print("cbt: ",ts[cbt_mins] + phi_ref)
        return ts[cbt_mins] + phi_ref
    
def dlmos(sol,show_plot):
        dlmo_convert = -7
        return cbt(sol,show_plot) + dlmo_convert # dlmo is defines by a relationship to cbt for this model


In [13]:
def clean_dlmo(dlmo_kj):
    import math
    type(dlmo_kj)
    np.set_printoptions(suppress=True,formatter={'float_kind':'{:f}'.format})
#     print("raw dlmos: ",dlmo_kj)
    dlmo_kj2 = []
    for i in range(len(dlmo_kj)):
        if dlmo_kj[i] <0:
            dlmo_kj2.append(24+dlmo_kj[i])
        elif i > 1:
            dlmo_kj2.append(dlmo_kj[i]-(24*(i-1)))
        else:
            dlmo_kj2.append(dlmo_kj[i])  

    '''based on the numbers of 24 hours we need to go back, 
    a ceiling function(in effect) is applied to make sure we always round up.Floor is used because the numbers
    are negative therefore a ceiling effect is produced by rounding downwards for negative numbers
    '''
#     print(np.array(dlmo_kj2))
    for i in range(len(dlmo_kj2)):
        if dlmo_kj2[i] < 0:
            dlmo_kj2[i] = (24*abs(math.floor(dlmo_kj2[i]/24)))+dlmo_kj2[i]
#     print(np.array(dlmo_kj2))
    return dlmo_kj2

In [14]:
def cvm(ip_time):
    TimeInMinutes = int(ip_time)*60+((ip_time%1)*60)
    return TimeInMinutes

import matplotlib.ticker as ticker

def Predicted_And_Actual(SleepingDates_ds, dlmo_kj2):

    sleep_48hr = list(SleepingDates_ds['start_time_num_round_25'])        
    #calculating estimated sleep based on 2hr gap between dlmo and sleep onset
    EstimatedSleepOnset = []

    for dlmo_time  in dlmo_kj2:
        if dlmo_time < 12:
            EstimatedSleepOnset.append(np.round(dlmo_time+24.0+2.0,2))
        else:
            EstimatedSleepOnset.append(np.round(dlmo_time+2.0,2))

    MinutesBetweenPredictedAndActualSleep_sleepwake = []
    for actual_time, est_time in zip(sleep_48hr,EstimatedSleepOnset):
        if not pd.isna(actual_time):
            MinutesBetweenPredictedAndActualSleep_sleepwake.append(np.round(cvm(actual_time)-cvm(est_time),2))
        else:
            MinutesBetweenPredictedAndActualSleep_sleepwake.append(None)

    PredictedVsActual = pd.DataFrame(zip(np.array(SleepingDates_ds['sleep_start_shift'].dt.date), 
                                                              np.array(SleepingDates_ds['start_time_num_round_25']),
                                                              EstimatedSleepOnset,
                                                   MinutesBetweenPredictedAndActualSleep_sleepwake))
    PredictedVsActual.columns=['sleep_start_shift','start_time_num_round','EstimatedSleepOnset',
                                  'MinBtwnActualPred']
#     print(len(SleepingDates_48hr['sleep_start_shift']),len(SleepingDates_48hr['start_time_num_round_25']),
#           len(EstimatedSleepOnset),len(dlmo_kj2))
        
    return PredictedVsActual


In [15]:
def phone_activity_usage(phone_sample,phoneContInnings_df,chosen_user,chosen_user_streak_end,
                         chosen_user_streak_start,streak_st, streak_end,measure_column):
    #        |--------------|
#     |-----------| ok
#                |-----------| ok
# |------------------------------|
#            |------| ok


    selected_phone_streaks = []
    _user_Phone = PhoneContInnings_df[(PhoneContInnings_df['user_id']== chosen_user) & 
                                                    (PhoneContInnings_df['count']>7)]
#     print("streak start: ",chosen_user_streak_start, " streak end: ", chosen_user_streak_end)
    _user_Phone = _user_Phone[~(_user_Phone['phone_streak_start_date'] > chosen_user_streak_end)]
    _user_Phone = _user_Phone[~(_user_Phone['phone_streak_end_date'] < chosen_user_streak_start)]
    _user_Phone = _user_Phone[['user_id','day_label','phone_streak_start_date','phone_streak_end_date',
                              'count']]
#     print("len og user phone: ",len(_user_Phone))
    if len(_user_Phone) == 0:
        return 0 
    else:    
        for i in range(len(_user_Phone)):
            if _user_Phone['phone_streak_start_date'].iloc[i]>= chosen_user_streak_start:
                if _user_Phone['phone_streak_end_date'].iloc[i] <= chosen_user_streak_end:
                    selected_phone_streaks.append([_user_Phone['phone_streak_start_date'].iloc[i],
                                                   _user_Phone['phone_streak_end_date'].iloc[i],
                                                   _user_Phone['count'].iloc[i]])
                elif _user_Phone['phone_streak_end_date'].iloc[i] > chosen_user_streak_end:

                    selected_phone_streaks.append([_user_Phone['phone_streak_start_date'].iloc[i],chosen_user_streak_end,
                                                   (chosen_user_streak_end-_user_Phone['phone_streak_start_date'].iloc[i]).days])

            elif _user_Phone['phone_streak_start_date'].iloc[i] < chosen_user_streak_start:
                if _user_Phone['phone_streak_end_date'].iloc[i] <= chosen_user_streak_end:
                    selected_phone_streaks.append([chosen_user_streak_start,_user_Phone['phone_streak_end_date'].iloc[i],
                                                   (_user_Phone['phone_streak_end_date'].iloc[i]-chosen_user_streak_start).days])

                elif _user_Phone['phone_streak_end_date'].iloc[i] > chosen_user_streak_end:
                    selected_phone_streaks.append([chosen_user_streak_start, chosen_user_streak_end,
                                                   chosen_user_streak_length])

        selected_phone_streaks_df = pd.DataFrame(selected_phone_streaks,columns=['phone_user_streak_start',
                                                                                  'phone_user_streak_end','count'])         
#         print("selected_phone_streaks_df: ",selected_phone_streaks_df)
        selected_phone_streaks_df['user_id'] = chosen_user

        selected_phone_streaks_df.sort_values(by='count',ascending=False,inplace=True)
#         print("length of selected_phone_streaks_df: ",len(selected_phone_streaks_df))
        
        if len(selected_phone_streaks_df) ==0:
            return 0
        else:

            phone_chosen_user_streak_start = selected_phone_streaks_df['phone_user_streak_start'].iloc[0]
            phone_chosen_user_streak_end = selected_phone_streaks_df['phone_user_streak_end'].iloc[0]

#             print("phone streak: ","\nstart: ",phone_chosen_user_streak_start, "\nend  : ",phone_chosen_user_streak_end)

            import math
            pp = phone_sample[(phone_sample["user_id"]==chosen_user) &
                              (phone_sample["day_label_dt"]>=  phone_chosen_user_streak_start) &
                              (phone_sample["day_label_dt"] <=  phone_chosen_user_streak_end)]
            pp = pp.reset_index()
            pp.drop(columns={"index"},inplace=True)
            '''rounding the start and end times of sleep to the nearest quarter for ease of calculation. will preserve
            sleep duration calculation'''
            pp['start_time_num_round_25'] = ((pp['start_time_num_round']%1)*100/60)+pp['start_time_num_round'].astype(int)
            
            
            ''' EXCEPTION HANDLING FOR SLEEP ONSET BETWEEN 23.45 AND 23.59 : SHOW AS 23.75 '''

            pp['start_time_num_round_25'] = round(pp['start_time_num_round_25']*4)/4
            pp['start_time_num_round_25'] = np.where(((pp['start_time_num_round'] >= 23.45) &
                                                       (pp['start_time_num_round'] < 24.0)), 
                                                     23.75,pp['start_time_num_round_25'])
        


            pp = pp[['day_label_dt','start_time_num_round_25',measure_column]].groupby(['day_label_dt','start_time_num_round_25']).sum().reset_index()
            pp.rename(columns={'day_label_dt':'day_label'},inplace=True)
            
            return pp

In [16]:
def PhoneActivityStreak(chosen_user, date_phone_activity_combo):
#     print("init length: ",len(date_phone_activity_combo))
    date_phone_activity_combo['hour_column'] = date_phone_activity_combo['start_time_num_round_25'].astype(int)
    date_phone_activity_combo['minute_column'] = (np.round(date_phone_activity_combo['start_time_num_round_25']%1,2)*60).astype(int)
    date_phone_activity_combo['time_strings'] = date_phone_activity_combo['hour_column'].astype(str).str.zfill(2)+':'+ \
    date_phone_activity_combo['minute_column'].astype(str).str.zfill(2)
    date_phone_activity_combo['time_column'] = pd.to_datetime(date_phone_activity_combo['time_strings'],format= '%H:%M').dt.time
    date_phone_activity_combo['sleep_start_datetime'] = pd.to_datetime(date_phone_activity_combo['sleep_start'].astype(str)+ ' '+
                                                          date_phone_activity_combo['time_column'].astype(str))

    date_phone_activity_combo.drop(columns={'duration','steps','dur_norm','steps_norm','hour_column',
                                           'minute_column','time_strings','time_column'},inplace=True)
    
#     print(date_phone_activity_combo.head())
    date_phone_activity_combo.sort_values(by=['sleep_start_datetime'],inplace=True)
    
    date_phone_activity_combo['lag_sleep_start_datetime'] = date_phone_activity_combo['sleep_start_datetime'].shift(1) #changed from -1

    conditions = [((date_phone_activity_combo['sleep_start_datetime']-date_phone_activity_combo['lag_sleep_start_datetime']).dt.days < 1.0) &
              ((date_phone_activity_combo['sleep_start_datetime']-
               date_phone_activity_combo['lag_sleep_start_datetime']).dt.seconds/60 == 15.0)] # change from ==1,switched around vars in subtraction
    choices = [1]
    date_phone_activity_combo['continuous_data_sleep'] = np.select(conditions,choices)

    s = date_phone_activity_combo['continuous_data_sleep'].eq(True)
    date_phone_activity_combo['count'] = (date_phone_activity_combo.groupby([s.ne(s.shift()).cumsum()])
                             .cumcount()
                             .add(1)
                            )
    date_phone_activity_combo_ContOnly = date_phone_activity_combo[date_phone_activity_combo['continuous_data_sleep']==1]
    
#     print("len of date_phone_activity_combo_ContOnly: ", len(date_phone_activity_combo_ContOnly))
    if len(date_phone_activity_combo_ContOnly)  < (96*7): # 6 days of data
        print("very small streak")
        return 0
    else:
        PhoneActivityContInnings = []
#         print("count of ones: ",len(date_phone_activity_combo_ContOnly[date_phone_activity_combo_ContOnly['count']==1]))
        if len(date_phone_activity_combo_ContOnly[date_phone_activity_combo_ContOnly['count']==1]) == 1:
#             print(date_phone_activity_combo_ContOnly['sleep_start'].iloc[-1],
#                  date_phone_activity_combo_ContOnly['count'].iloc[-1])
            PhoneActivityContInnings.append([chosen_user, 
                                             date_phone_activity_combo_ContOnly['sleep_start'].iloc[-1],
                                             date_phone_activity_combo_ContOnly['count'].iloc[-1]])
        else:
            for i in range(2,len(date_phone_activity_combo_ContOnly)):
                if date_phone_activity_combo_ContOnly['count'].iloc[i] == 1:
                    PhoneActivityContInnings.append([chosen_user,
                                                     date_phone_activity_combo_ContOnly['sleep_start'].iloc[i-1],
                                                     date_phone_activity_combo_ContOnly['count'].iloc[i-1]])

            #adding the last streak, however long or short it is
            PhoneActivityContInnings.append([chosen_user,
                                             date_phone_activity_combo_ContOnly['sleep_start'].iloc[-1],
                                             date_phone_activity_combo_ContOnly['count'].iloc[-1]])
        
        if len(PhoneActivityContInnings)== 0:
            print("no continuous streak found")
            return 0
        else:
            PhoneActivityContInnings_df = pd.DataFrame(PhoneActivityContInnings,columns =['user_id','sleep_start','count'])
            PhoneActivityContInnings_df = PhoneActivityContInnings_df[PhoneActivityContInnings_df['count']>1]
            PhoneActivityContInnings_df['phone_activity_streak_start_date']=PhoneActivityContInnings_df['sleep_start']-\
            pd.to_timedelta((PhoneActivityContInnings_df['count']/96).astype(int),unit='d')
            PhoneActivityContInnings_df.rename(columns={'sleep_start':'phone_activity_streak_end_date'},inplace=True)
            PhoneActivityContInnings_df['count'] = (PhoneActivityContInnings_df['count']/96).astype(int)
            # PhoneActivityContInnings_df.head()
            PhoneActivityContInnings_df.sort_values(by='count',ascending=False,inplace= True)
            PhoneActivityContInnings_df2 = PhoneActivityContInnings_df.drop_duplicates('user_id',
                                                                                       keep = 'first')
#             print(PhoneActivityContInnings_df2)
            #retaining only the longest streak

            streak_start = PhoneActivityContInnings_df2['phone_activity_streak_start_date'].values[0]
            streak_end = PhoneActivityContInnings_df2['phone_activity_streak_end_date'].values[0]
#             print("streak start: ",streak_start, " streak end: ", streak_end)

            date_phone_activity_combo = date_phone_activity_combo[(date_phone_activity_combo['sleep_start']>= streak_start) &
                                                                 (date_phone_activity_combo['sleep_start']<= streak_end)]

            return date_phone_activity_combo


In [19]:
def GetStreak(chosen_user, date_combo, date_sleep_combo, measure_column):
    date_combo['hour_column'] = date_combo['start_time_num_round_25'].astype(int)
    date_combo['minute_column'] = (np.round(date_combo['start_time_num_round_25']%1,2)*60).astype(int)
    date_combo['time_strings'] = date_combo['hour_column'].astype(str).str.zfill(2)+':'+ date_combo['minute_column'].astype(str).str.zfill(2)
    date_combo['time_column'] = pd.to_datetime(date_combo['time_strings'],format= '%H:%M').dt.time
    date_combo['sleep_start_datetime'] = pd.to_datetime(date_combo['sleep_start'].astype(str)+ ' '+
                                                          date_combo['time_column'].astype(str))

    date_combo.drop(columns={'hour_column','minute_column','time_strings','time_column'},inplace=True)
    date_combo.sort_values(by=['sleep_start_datetime'],inplace=True)
    
    date_combo['lag_sleep_start_datetime'] = date_combo['sleep_start_datetime'].shift(1) #changed from -1

    conditions = [((date_combo['sleep_start_datetime']-date_combo['lag_sleep_start_datetime']).dt.days < 1.0) &
              ((date_combo['sleep_start_datetime']-
               date_combo['lag_sleep_start_datetime']).dt.seconds/60 == 15.0)] # change from ==1,switched around vars in subtraction
    choices = [1]
    date_combo['continuous_data_sleep'] = np.select(conditions,choices)

    s = date_combo['continuous_data_sleep'].eq(True)
    date_combo['count'] = (date_combo.groupby([s.ne(s.shift()).cumsum()])
                             .cumcount()
                             .add(1)
                            )
    date_combo_ContOnly = date_combo[date_combo['continuous_data_sleep']==1]
    
#     return date_combo_ContOnly
    print("len of date_combo_ContOnly: ", len(date_combo))
    if len(date_combo)  < (96*7): # 6 days of data
        return 0
    else:
        _ContInnings = []
        if len(date_combo_ContOnly[date_combo_ContOnly['count']==1]) == 1:
            _ContInnings.append([chosen_user, date_combo_ContOnly['sleep_start'].iloc[-1],
                                 date_combo_ContOnly['count'].iloc[-1]])
        else:
            for i in range(2,len(date_combo_ContOnly)):
                if date_combo_ContOnly['count'].iloc[i] == 1:
                    _ContInnings.append([chosen_user,
                                                     date_combo_ContOnly['sleep_start'].iloc[i-1],
                                                     date_combo_ContOnly['count'].iloc[i-1]])
 
          
            _ContInnings.append([chosen_user,
                                 date_combo_ContOnly['sleep_start'].iloc[-1],
                                 date_combo_ContOnly['count'].iloc[-1]])
        
        if len(_ContInnings)== 0:
            return 0
        else:
            _ContInnings_df = pd.DataFrame(_ContInnings,columns =['user_id','sleep_start','count'])
            _ContInnings_df = _ContInnings_df[_ContInnings_df['count']>1]
            _ContInnings_df['phone_activity_streak_start_date']=_ContInnings_df['sleep_start']-\
            pd.to_timedelta((_ContInnings_df['count']/96).astype(int),unit='d')
            _ContInnings_df.rename(columns={'sleep_start':'phone_activity_streak_end_date'},inplace=True)
            _ContInnings_df['count'] = (_ContInnings_df['count']/96).astype(int)
            # PhoneActivityContInnings_df.head()
            _ContInnings_df.sort_values(by='count',ascending=False,inplace= True)

            _ContInnings_df2 = _ContInnings_df.drop_duplicates('user_id', keep = 'first')
            #retaining only the longest streak

            streak_start = _ContInnings_df2['phone_activity_streak_start_date'].values[0]
            streak_end = _ContInnings_df2['phone_activity_streak_end_date'].values[0]

            date_combo = date_combo[(date_combo['sleep_start']>= streak_start) &
                                    (date_combo['sleep_start']<= streak_end)]
#             print("len of date combo: ",len(date_combo))
            if measure_column == 'duration':
                from sklearn.preprocessing import MinMaxScaler
                scaler = MinMaxScaler()
                a = np.array(date_combo['duration']).reshape(-1,1)
                a1 = scaler.fit_transform(a)
                a1 = pd.Series(a1.flatten(),name='dur_norm')
                date_combo = pd.concat([date_combo.reset_index(drop=True),a1.reset_index(drop=True)],axis=1)
                date_combo['dur_norm'] = date_combo['dur_norm']*10000
                
                date_combo = date_sleep_combo.merge(date_combo[['sleep_start','start_time_num_round','dur_norm']],
                                                       how='inner',
                                                       left_on = ['sleep_start','start_time_num_round'],
                                                      right_on = ['sleep_start','start_time_num_round'])
                date_combo['dur_norm']=np.where(date_combo['asleep_0_awake_300'] == 0,0,
                                               date_combo['dur_norm'])
        
#                 print("count of missings: ", date_combo['dur_norm'].isna().sum())
                
#                 print("date combo cols: ", date_combo.columns)
            else:
                from sklearn.preprocessing import MinMaxScaler
                scaler = MinMaxScaler()
                a = np.array(date_combo['steps']).reshape(-1,1)
                a1 = scaler.fit_transform(a)
                a1 = pd.Series(a1.flatten(),name='steps_norm')
                date_combo = pd.concat([date_combo.reset_index(drop=True),a1.reset_index(drop=True)],axis=1)
                date_combo['steps_norm'] = date_combo['steps_norm']*10000

                date_combo = date_sleep_combo.merge(date_combo[['sleep_start','start_time_num_round','steps_norm']],
                                                       how='inner',
                                                       left_on = ['sleep_start','start_time_num_round'],
                                                      right_on = ['sleep_start','start_time_num_round'])
                date_combo['steps_norm']=np.where(date_combo['asleep_0_awake_300'] == 0,0,
                                               date_combo['steps_norm'])
                
                print("count of missings: ", date_combo['steps_norm'].isna().sum())
                print(date_combo)
#                 print("date combo cols: ", date_combo.columns)


    return date_combo
